<a href="https://colab.research.google.com/github/zain910128/MS-Data_Mining-Identity_Matching/blob/master/Data_Mining_Project_Identity_Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Executive Summary



### Problem statement:

The dataset contains 1,000,000 records of patient information. 

The objective is to find the records / "enterprise id" which belong to the same person using only their demographic data

### Significance of the problem:

- With increase in adoption of EHR system and improvement in Health IT infrastructure across the system, there is a more than ever increasing need for integrating profiles and data across multiple systems
- It is critical to match profiles to create a comprehensive history of patient’s health record and to improve interoperability in the system
- Health care providers must be able to share patient health information and accurately match a patient to his or her data from a different provider
- Matching mistakes can contribute toward adverse events, compromised safety and privacy, and increased health care costs due to repeat tests, and other factors
- The cost to manually correct mismatched patient records is estimated to be $60 per record not including the potential harm that could be caused due to a patient receiving the wrong treatment and potential legal fees

### Data source:
  - https://www.patientmatchingchallenge.com/challenge-information/challenge-details

  - https://github.com/onc-healthit/patient-matching


### Important resources:

  - https://mindfulmachines.io/blog/2017/7/23/onc-patient-matching-challenge-part-2
  - https://people.cs.pitt.edu/~kirk/cs1501/Pruhs/Spring2006/assignments/editdistance/Levenshtein%20Distance.htm

  - https://github.com/oakmac/record-linking-talk/blob/master/src/com/oakmac/conj2019/record_linking_talk/core.clj
  - https://www.youtube.com/watch?v=rGKEOMUtJfE
  - http://www.bristol.ac.uk/media-library/sites/cmm/migrated/documents/problinkage.pdf (slide 17 onwards)
  - https://www.ncbi.nlm.nih.gov/books/NBK253312/
  - https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536

# Get Data and Pre-Process

In [46]:
import pandas as pd
import io
import requests
import numpy as np

urls=["https://raw.githubusercontent.com/onc-healthit/patient-matching/master/ONC%20Patient%20Matching%20Algorithm%20Challenge%20Test%20Dataset.A-C.csv",
     "https://raw.githubusercontent.com/onc-healthit/patient-matching/master/ONC%20Patient%20Matching%20Algorithm%20Challenge%20Test%20Dataset.D-F.csv",
     "https://raw.githubusercontent.com/onc-healthit/patient-matching/master/ONC%20Patient%20Matching%20Algorithm%20Challenge%20Test%20Dataset.G-I.csv",
     "https://raw.githubusercontent.com/onc-healthit/patient-matching/master/ONC%20Patient%20Matching%20Algorithm%20Challenge%20Test%20Dataset.J-mid%20L.csv",
     "https://raw.githubusercontent.com/onc-healthit/patient-matching/master/ONC%20Patient%20Matching%20Algorithm%20Challenge%20Test%20Dataset.mid%20L%20-%20N.csv",
     "https://raw.githubusercontent.com/onc-healthit/patient-matching/master/ONC%20Patient%20Matching%20Algorithm%20Challenge%20Test%20Dataset.O-R.csv",
     "https://raw.githubusercontent.com/onc-healthit/patient-matching/master/ONC%20Patient%20Matching%20Algorithm%20Challenge%20Test%20Dataset.S.csv",
     "https://raw.githubusercontent.com/onc-healthit/patient-matching/master/ONC%20Patient%20Matching%20Algorithm%20Challenge%20Test%20Dataset.T-Z.csv",
     "https://raw.githubusercontent.com/onc-healthit/patient-matching/master/ONC%20Patient%20Matching%20Algorithm%20Challenge%20Test%20Dataset.Null.csv",
    ]

data = pd.DataFrame()
for url in urls:
  s=requests.get(url).content
  data = pd.concat([data , pd.read_csv(io.StringIO(s.decode('utf-8')))])

print(data.shape)
print(data.dtypes)
data= data.reset_index()
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1000000, 19)
EnterpriseID             int64
LAST                    object
FIRST                   object
MIDDLE                  object
SUFFIX                  object
DOB                    float64
GENDER                  object
SSN                     object
ADDRESS1                object
ADDRESS2                object
ZIP                     object
MOTHERS_MAIDEN_NAME     object
MRN                    float64
CITY                    object
STATE                   object
PHONE                   object
PHONE2                  object
EMAIL                   object
ALIAS                   object
dtype: object


,index,EnterpriseID,LAST,FIRST,MIDDLE,SUFFIX,DOB,GENDER,SSN,ADDRESS1,ADDRESS2,ZIP,MOTHERS_MAIDEN_NAME,MRN,CITY,STATE,PHONE,PHONE2,EMAIL,ALIAS
0,0,14065387,AABERG,KATHERINE,NaN,NaN,8476.00000,FEMALE,892-39-5115,1414 HAZEN ST,NaN,11370,NaN,nan,FLUSHING,NY,347-984-6839,NaN,NaN,NaN
1,1,12847713,AABY,PATSY,NaN,NaN,12222.00000,FEMALE,NaN,1621 UN ST,1E,11213,NaN,4723376.00000,BROOKLYN,NY,917-130-8285,NaN,NaN,NaN
2,2,12263824,AACANC,NaN,NaN,NaN,31887.00000,MALE,843-26-7213,9427 KINGS HWY,6F,11223,NaN,nan,BROOKLYN,NY,201-531-2857,NaN,NaN,AACANC
3,3,14781139,AADAHLGEBERT,ARLEEN,NaN,NaN,16007.00000,FEMALE,NaN,620 TRINITY AVE,D16,10455,NaN,nan,BRONX,NY,732-950-2944,NaN,AAADAHLGEBERT@AMGGT.COM,NaN
4,4,15616535,AADDAT,PIERCE,RALPH,NaN,14730.00000,M,NaN,1155 MORRISON AVENUE,1E,10472,NaN,4859518.00000,BRONX,NY,NaN,NaN,NaN,PIERCE RALPH AADDAT


# EDA

In [47]:
data.isnull().sum()

index                       0
EnterpriseID                0
LAST                    33257
FIRST                   33884
MIDDLE                 617710
SUFFIX                 990953
DOB                     16463
GENDER                  10059
SSN                    244606
ADDRESS1                68653
ADDRESS2               309639
ZIP                     71607
MOTHERS_MAIDEN_NAME    950828
MRN                    536958
CITY                    72716
STATE                   73687
PHONE                   21876
PHONE2                 853670
EMAIL                  764149
ALIAS                  945867
dtype: int64

In [48]:
data.apply(lambda x: x.duplicated(keep=False).sum())

index                   937848
EnterpriseID                 0
LAST                    900850
FIRST                   965219
MIDDLE                  994009
SUFFIX                 1000000
DOB                     999775
GENDER                 1000000
SSN                     896583
ADDRESS1                763680
ADDRESS2                995037
ZIP                     992900
MOTHERS_MAIDEN_NAME     984911
MRN                     536958
CITY                    997836
STATE                   999986
PHONE                    82562
PHONE2                  898715
EMAIL                   894009
ALIAS                   960264
dtype: int64

# Data Cleaning

-	Gender is available for 99% records
-	Age: Data consists of at least 100k records for each age group with slightly lower data for the age group of 80-100-year-old.
-	Suffix: The most common suffixes are: II, Sr, Jr. However, not a lot of records have suffixes
-	Name and Last Name are available for almost all records, but we checked if there were any special characters in these fields.
-	ZIP: The entries were of varying lengths. Upon closer inspection we found that 2 character entries were State codes, 7 character entries were float, and 12 character ones were phone numbers
-	Phone: 12 character entries had hyphens, some phone numbers were invalid entries such as “1111111111”. 7 Character entries had the area code missing.
-	Phone2: This column was used to capture additional phone numbers, and thus contained entries having up to 3 phone numbers separated by “^^”
-	Email: Interestingly, all emails were on the same domain, “amggt.com”. A lot of email addresses were invalid with just one character username. Due to poor quality of data, this may not be a useful column.
-	ALIAS: This has 95% null values, so we will drop this.
-	State: All entries were 2 character state codes. Some state codes were invalid, such as “zz”. Whereas some were possible typing errors, like “CD” instead of “DC”
-	City: City names were available for all addresses however we noticed spelling errors. We may need to correct spellings in order to use this column effectively.
-	Address1: This was available for all records and contains the street address
-	Address2: Contained house number. There is no way to correct any typing errors and will be taken at face value
-	Enterprise ID: This was a unique ID assigned to each record and there are no duplicates.
-	SSN: It was available for 75% users. However, there are some duplicates
-	MRN: Medical record number was assigned to users, it was supposed to be unique, however, we needed to verify that.


## FIRST, LAST & MOTHER'S NAME

In [49]:
import re 
string_check= re.compile('[@_!#$%^&*()<>?/\|}{~:]') # Cheking for special characters
data[data['FIRST'].str.contains(string_check,na=False)]
data[data['LAST'].str.contains(string_check,na=False)]
data[data['MOTHERS_MAIDEN_NAME'].str.contains(string_check,na=False)]

,index,EnterpriseID,LAST,FIRST,MIDDLE,SUFFIX,DOB,GENDER,SSN,ADDRESS1,ADDRESS2,ZIP,MOTHERS_MAIDEN_NAME,MRN,CITY,STATE,PHONE,PHONE2,EMAIL,ALIAS


## SUFFIX



In [50]:
#Suffix
#data.groupby(by='SUFFIX').size()   # length 3 = NaN
data["SUFFIX"].replace("JR." , 'JR', inplace = True)
data["SUFFIX"].replace("SR." , 'SR', inplace = True)

## GENDER



In [51]:
# GENDER
# 5 different values: M,F, MALE, FEMALE, U; reducing them to 3 (M,F,U)

#data.groupby(by='GENDER').size()
data["GENDER"].replace("MALE" , 'M', inplace = True)
data["GENDER"].replace("FEMALE" , 'F', inplace = True)

# ideas to explore: less weight for gender if Unknown or maybe figuring out gender by name

## ZIP

In [52]:
# to remove scientific notation:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
# to reset:
#pd.reset_option('display.float_format')

In [53]:
set(data['ZIP'].astype('str').apply(len))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

In [54]:
data.loc[ (data['ZIP'].astype('str').apply(len) == 7 ),['ZIP'] ]

,ZIP
631072,11370.00000
631073,10452.00000
631074,11037.00000
631075,11717.00000
631076,10451.00000
...,...
999995,11205.00000
999996,11725.00000
999997,11780.00000
999998,10452.00000


In [55]:
# 12 character zip codes seem to be phone numbers with hyphens

data['PHONE_ZIP'] = data.loc[ (data['ZIP'].astype('str').apply(len) == 12 ) , ['ZIP'] ]
data['PHONE_ZIP'] = data['PHONE_ZIP'].str.replace(r'[^0-9]+', '')

In [56]:
# Keeping zipcodes that can be cleaned up and nullifying all others
data.loc[ (data['ZIP'].astype('str').apply(len) != 7 ) & 
         (data['ZIP'].astype('str').apply(len) != 9 ) & 
         (data['ZIP'].astype('str').apply(len) != 10 ) & 
         (data['ZIP'].astype('str').apply(len) != 11 ) & 
         (data['ZIP'].astype('str').apply(len) != 5 ),
         ['ZIP'] ] = np.nan

# 7 digit zip codes are 5 digit zip codes but with a decimal place. Convert to int.
data.loc[ (data['ZIP'].astype('str').apply(len) == 7 ),['ZIP'] ] = \
data.loc[ (data['ZIP'].astype('str').apply(len) == 7 ),['ZIP'] ].astype('int')

# 9, 10, 11 digit zip codes seem to have an extension which can be trimmed out 
# by taking 5 characters from left after converting to int
data.loc[ (data['ZIP'].astype('str').apply(len) == 9 ) | 
         (data['ZIP'].astype('str').apply(len) == 10 ) | 
         (data['ZIP'].astype('str').apply(len) == 11 ) ,
         ['ZIP'] ] = data.loc[ (data['ZIP'].astype('str').apply(len) == 9 ) | 
         (data['ZIP'].astype('str').apply(len) == 10 ) | 
         (data['ZIP'].astype('str').apply(len) == 11 ) ,
         ['ZIP'] ].apply(lambda x: str(x[0])[0:5],axis=1)

# Define column dtype
data.loc[:,'ZIP'] = data.loc[:,'ZIP'].astype('str')

## PHONE

In [57]:
set(data['PHONE'].dropna().apply(len))

{1, 8, 10, 12}

In [58]:
# Remove hyphens
data['PHONE'] = data['PHONE'].replace("-","",regex=True)

In [59]:
set(data['PHONE'].dropna().apply(len))

{1, 7, 10}

In [60]:
data.loc[data['PHONE'].astype('str').apply(len)==7,['PHONE']]

,PHONE
162981,8963452
412054,1111111
412055,1111111
637077,8963452
660312,7759582
660313,3456544
660314,3586357
660316,7759582
660321,3586357
660322,3456544


In [61]:
# Setting 1 numbers to nulls and retaining 7 digit numbers as they may be valid numbers without area code.

data.loc[data['PHONE'].astype('str').apply(len)==1,['PHONE']] = np.nan

In [62]:
data['PHONE'] = data['PHONE'].str.replace(r'[^0-9]+', '')

## PHONE2

In [63]:
set(data['PHONE2'].dropna().apply(len))

{12, 26, 40}

In [64]:
sum(data['PHONE2'].dropna().apply(len)>12)

6205

In [65]:
data[['PHONE2a','PHONE2b','PHONE2c']] = data['PHONE2'][data['PHONE2'].astype('str').apply(len)>12].str.split('\^{2}', expand=True) # '^^' canot be passed into split because the method runs on regex where ^^ has a different meaning.

In [66]:
set(data['PHONE2a'].dropna().astype('str').apply(len))

{12}

In [67]:
data[['PHONE2c']][data['PHONE2c'].notna()]

,PHONE2c
96818,848-210-5397
170586,862-945-2928
441732,860-316-3987
518864,267-495-4061
603005,848-212-6310
789300,551-144-9669
873843,732-637-2207


In [68]:
data['PHONE2a'] = data['PHONE2a'].str.replace(r'[^0-9]+', '')
data['PHONE2b'] = data['PHONE2b'].str.replace(r'[^0-9]+', '')
data['PHONE2c'] = data['PHONE2c'].str.replace(r'[^0-9]+', '')

data[['PHONE2c']][data['PHONE2c'].notna()].head()

,PHONE2c
96818,8482105397
170586,8629452928
441732,8603163987
518864,2674954061
603005,8482126310


## EMAIL 

In [69]:
data.EMAIL.count()

235851

In [70]:
data.EMAIL.str.endswith("AMGGT.COM").sum()

235851

In [71]:
data['eid'] = data.EnterpriseID
e = data[['EMAIL','eid']].groupby('EMAIL').count()

In [72]:
e[[e>1][0]['eid']].sort_values(by='eid', ascending=False)

,eid
EMAIL,
M@AMGGT.COM,1192
J@AMGGT.COM,938
A@AMGGT.COM,877
S@AMGGT.COM,838
C@AMGGT.COM,809
...,...
JGLAZER@AMGGT.COM,2
JGISOMAR@AMGGT.COM,2
JGILLEY@AMGGT.COM,2


In [73]:
data.groupby('EMAIL').get_group('AAARON@AMGGT.COM')

,index,EnterpriseID,LAST,FIRST,MIDDLE,SUFFIX,DOB,GENDER,SSN,ADDRESS1,ADDRESS2,ZIP,MOTHERS_MAIDEN_NAME,MRN,CITY,STATE,PHONE,PHONE2,EMAIL,ALIAS,PHONE_ZIP,PHONE2a,PHONE2b,PHONE2c,eid
68,68,13771070,AARON,ADI,NaN,NaN,33203.00000,F,NaN,1070 OGDEN AVE,2A,10452,NaN,2600143.00000,BRONX,NY,4841043683,NaN,AAARON@AMGGT.COM,NaN,NaN,NaN,NaN,NaN,13771070
83,83,14772302,AARON,ALTA,R,NaN,10842.00000,F,NaN,2061 W 4TH ST,1,11223,NaN,4241713.00000,BROOKLYN,NY,8603217374,NaN,AAARON@AMGGT.COM,NaN,NaN,NaN,NaN,NaN,14772302
95,95,15234184,AARON,AYANA,NaN,NaN,8120.00000,F,860-56-5541,219 HAWTHORNE AVENUE APT 415,NaN,11722,NaN,nan,CENTRAL ISLIP,NY,5167448898,NaN,AAARON@AMGGT.COM,NaN,NaN,NaN,NaN,NaN,15234184


## STATE

In [74]:
set(data['STATE'].astype('str').apply(len))

{1, 2, 3, 5, 6, 8}

In [75]:
us_states = {'AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NA', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY'}

In [76]:
set(data['STATE'][data['STATE'].astype('str').apply(len)==2]) - us_states

{'CD', 'CN', 'MY', 'ON', 'OS', 'QC', 'UN', 'ZZ'}

In [77]:
data[['STATE']][ (data['STATE'].astype('str').apply(len) == 5 )].dropna()

,STATE
182297,BRONX
602925,10454


In [78]:
import numpy as np

## CITY

In [79]:
set(data['CITY'].astype('str').apply(len))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22}

In [80]:
data[['CITY']][(data['CITY'].astype('str').apply(len) == 9 )].dropna()

,CITY
61,WOODRIDGE
88,LAURELTON
107,ROCHESTER
109,BAY SHORE
110,ST ALBANS
...,...
999958,RIVERHEAD
999959,RIVERHEAD
999969,ST ALBANS
999978,ST.ALBANS


In [81]:
def clean_gender(x):
    if x in ('FEMALE','F'):
        return 'F'
    if x in ('MALE','M'):
        return 'M'
    else:
        return 'U'

In [82]:
data['GENDER'] = data['GENDER'].apply(clean_gender)

## ADDRESS1

## ADDRESS2

# Creating Blocking Key


In [83]:
data["BLOCK"] = data["FIRST"].astype(str).str[0:2] + data["LAST"].astype(str).str[0:2] + data["GENDER"] + data["DOB"].astype(str).str[-4:-2]

# Record Linkage 

In [84]:
data.BLOCK.value_counts()

MAnaFn     37
MAnaF79    28
MAnaF38    25
JOMAM96    25
MAnaF60    24
           ..
ANKIF49     1
STVAM05     1
DALOM34     1
MOKHU45     1
AMXEF13     1
Name: BLOCK, Length: 669224, dtype: int64

In [44]:
!pip install recordlinkage

     |████████████████████████████████| 952kB 2.7MB/s 
     |████████████████████████████████| 102kB 7.1MB/s 


In [99]:
d

,index,EnterpriseID,LAST,FIRST,MIDDLE,SUFFIX,DOB,GENDER,SSN,ADDRESS1,ADDRESS2,ZIP,MOTHERS_MAIDEN_NAME,MRN,CITY,STATE,PHONE,PHONE2,EMAIL,ALIAS,PHONE_ZIP,PHONE2a,PHONE2b,PHONE2c,eid,BLOCK
0,0,14065387,AABERG,KATHERINE,NaN,NaN,8476.00000,F,892-39-5115,1414 HAZEN ST,NaN,11370,NaN,nan,FLUSHING,NY,3479846839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14065387,KAAAF76
1,1,12847713,AABY,PATSY,NaN,NaN,12222.00000,F,NaN,1621 UN ST,1E,11213,NaN,4723376.00000,BROOKLYN,NY,9171308285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12847713,PAAAF22
2,2,12263824,AACANC,NaN,NaN,NaN,31887.00000,M,843-26-7213,9427 KINGS HWY,6F,11223,NaN,nan,BROOKLYN,NY,2015312857,NaN,NaN,AACANC,NaN,NaN,NaN,NaN,12263824,naAAM87
3,3,14781139,AADAHLGEBERT,ARLEEN,NaN,NaN,16007.00000,F,NaN,620 TRINITY AVE,D16,10455,NaN,nan,BRONX,NY,7329502944,NaN,AAADAHLGEBERT@AMGGT.COM,NaN,NaN,NaN,NaN,NaN,14781139,ARAAF07
4,4,15616535,AADDAT,PIERCE,RALPH,NaN,14730.00000,M,NaN,1155 MORRISON AVENUE,1E,10472,NaN,4859518.00000,BRONX,NY,NaN,NaN,NaN,PIERCE RALPH AADDAT,NaN,NaN,NaN,NaN,15616535,PIAAM30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,99996,15908742,BROCKHAUS,SUSAN,NELSON,NaN,19526.00000,F,855-43-8205,2120 RANDALE AVENUE,2A,10473,NaN,4841936.00000,BRONX,NY,9349223835,973-664-5261,NaN,NaN,NaN,NaN,NaN,NaN,15908742,SUBRF26
99997,99997,12739899,BROCKHOFF,CHRISTINA,LINDA,NaN,39976.00000,F,870-21-5892,484 EAST 96TH ST,PVT,11212,NaN,nan,BROOKLYN,NY,6314715877,NaN,CBROCKHOFF@AMGGT.COM,NaN,NaN,NaN,NaN,NaN,12739899,CHBRF76
99998,99998,14980744,BROCKHOFF,CHRISTINA,NaN,NaN,8777.00000,F,840-84-5485,2928 HONE AVE,PH,10467,NaN,nan,BRONX,NY,9343704365,NaN,CBROCKHOFF@AMGGT.COM,NaN,NaN,NaN,NaN,NaN,14980744,CHBRF77
99999,99999,12884956,BROCKHOUSE,SYLVIA,STELLA,NaN,38719.00000,F,NaN,MABON BUILDING,13,10035,NaN,nan,NEW YORK,NY,6098986989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12884956,SYBRF19


In [113]:
import recordlinkage as rl
from recordlinkage.index import Block
from recordlinkage.compare import Exact, String

# running on subset of data
d = data.loc[0:100000]

#Creating comparison pairs using blocking key
indexer = rl.BlockIndex(on='BLOCK')
pairs = indexer.index(d)
print (len(pairs))

# Calculating distance matrix for selected features
comparer = rl.Compare()
#comparer.add(String('DOB', 'DOB', label='DOB',threshold=0.95,method='damerau_levenshtein'))
#comparer.add(Exact('GENDER', 'GENDER', label='GENDER'))
comparer.add(String('FIRST', 'FIRST', threshold=0.90,method='damerau_levenshtein', label='FIRST'))
comparer.add(String('LAST', 'LAST', threshold=0.90,method='damerau_levenshtein',label='LAST'))
comparison1 = comparer.compute(pairs, d[['FIRST','LAST']])
print('feature shape', comparison1.shape)

# Criterion for matching pairs
matches = comparison1[comparison1.sum(axis=1) >= 3]
print(matches.shape)

52856
feature shape (52856, 2)
(0, 2)


In [114]:
comparison1

,,FIRST,LAST
100,99,1.00000,1.00000
105,101,1.00000,1.00000
104,102,1.00000,1.00000
106,103,1.00000,1.00000
145,144,1.00000,1.00000
...,...,...,...
99970,99957,1.00000,1.00000
99974,99959,1.00000,1.00000
99965,99961,1.00000,1.00000
99968,99964,1.00000,1.00000


# Fellegi Sunter Method - Unsupervised

In [85]:
# !pip install recordlinkage

import recordlinkage as rl

In [86]:
matching_data = data.loc[:2500,['LAST','FIRST','GENDER','SSN','CITY','PHONE']]

In [87]:
matching_data["BLOCK_SN"] = matching_data["FIRST"].astype(str) + matching_data["LAST"].astype(str) + matching_data["SSN"].astype(str) + matching_data["CITY"].astype(str)  + data["GENDER"].astype(str) + data["PHONE"].astype(str)

indexer = rl.SortedNeighbourhoodIndex(
        'BLOCK_SN',window=3, )
pairs = indexer.index(matching_data)
pairs

MultiIndex([(  52,    7),
            (  57,   31),
            (  63,   44),
            (  95,   65),
            ( 105,  101),
            ( 106,  103),
            ( 117,  116),
            ( 156,   85),
            ( 169,  111),
            ( 181,  155),
            ...
            (2491,  730),
            (2492, 1168),
            (2493, 2233),
            (2494, 2493),
            (2495, 2475),
            (2496, 2387),
            (2497, 1027),
            (2498, 2471),
            (2499,  322),
            (2500, 2495)],
           length=2544)

In [88]:
comparer = rl.Compare()
comparer.string('LAST', 'LAST', method='jarowinkler', threshold=0.85, label='LAST')
comparer.string('FIRST', 'FIRST', method='jarowinkler', threshold=0.85, label='FIRST')
comparer.string('GENDER', 'GENDER', method='jarowinkler', threshold=0.85, label='GENDER')
comparer.string('SSN', 'SSN', method='jarowinkler', threshold=0.85, label='SSN')
comparer.string('CITY', 'CITY', method='jarowinkler', threshold=0.85, label='CITY')
comparer.string('PHONE', 'PHONE', method='jarowinkler', threshold=0.85, label='PHONE')
comparison = comparer.compute(pairs, matching_data)
comparison

,,LAST,FIRST,GENDER,SSN,CITY,PHONE
52,7,0.00000,1.00000,1.00000,0.00000,0.00000,0.00000
57,31,0.00000,1.00000,1.00000,0.00000,0.00000,0.00000
63,44,1.00000,1.00000,0.00000,0.00000,0.00000,0.00000
95,65,1.00000,1.00000,1.00000,0.00000,0.00000,0.00000
105,101,1.00000,1.00000,1.00000,1.00000,1.00000,0.00000
...,...,...,...,...,...,...,...
2496,2387,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000
2497,1027,0.00000,1.00000,1.00000,0.00000,0.00000,0.00000
2498,2471,0.00000,1.00000,1.00000,0.00000,1.00000,0.00000
2499,322,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [89]:



# Initialise the Expectation-Conditional Maximisation classifier.
cl = rl.ECMClassifier()
cl.fit(comparison)

# Print the parameters that are trained (m, u and p). Note that the estimates
# are very good.
print("p probability P(Match):", cl.p)
print("m probabilities P(x_i=1|Match):", cl.m_probs)
print("u probabilities P(x_i=1|Non-Match):", cl.u_probs)
print("log m probabilities P(x_i=1|Match):", cl.log_m_probs)
print("log u probabilities P(x_i=1|Non-Match):", cl.log_u_probs)
print("log weights of features:", cl.log_weights)
print("weights of features:", cl.weights)

# evaluate the model
links_pred = cl.predict(comparison)
print("Predicted number of links:", len(links_pred))


probs = cl.prob(comparison)

p probability P(Match): 0.10826028393681564
m probabilities P(x_i=1|Match): {'LAST': {0.0: 6.641654872547067e-07, 1.0: 0.9999993358345124}, 'FIRST': {0.0: 3.986111124978236e-14, 1.0: 0.99999999999996}, 'GENDER': {0.0: 0.057835284111844035, 1.0: 0.9421647158881559}, 'SSN': {0.0: 0.8004761467170443, 1.0: 0.19952385328295535}, 'CITY': {0.0: 0.5862941476200279, 1.0: 0.4137058523799718}, 'PHONE': {0.0: 0.7785153839483917, 1.0: 0.22148461605160769}}
u probabilities P(x_i=1|Non-Match): {'LAST': {0.0: 0.8745535584942769, 1.0: 0.1254464415057235}, 'FIRST': {0.0: 0.46548822727687694, 1.0: 0.5345117727231234}, 'GENDER': {0.0: 0.33019307942255394, 1.0: 0.6698069205774466}, 'SSN': {0.0: 0.9995379037174758, 1.0: 0.0004620962825241195}, 'CITY': {0.0: 0.8602391216812181, 1.0: 0.13976087831878287}, 'PHONE': {0.0: 1.0, 1.0: 3.153587298208812e-39}}
log m probabilities P(x_i=1|Match): {'LAST': {0.0: -14.224734490728832, 1.0: -6.641657082084862e-07}, 'FIRST': {0.0: -30.853375201694213, 1.0: -3.996802888650

In [90]:
probs[probs>.98]

105   101    0.99961
106   103    0.99829
117   116    0.99684
217   216    1.00000
288   285    1.00000
               ...  
2331  2321   0.99961
2334  2333   1.00000
2385  2384   0.99829
2427  2426   1.00000
2494  2493   0.99684
Length: 105, dtype: float64

# Checking a few matched records to see if they match

In [91]:
matching_data.iloc[[105,101],:]

,LAST,FIRST,GENDER,SSN,CITY,PHONE,BLOCK_SN
105,AARON,BRENT,M,891-71-5745,FLUSHING,2128679131,BRENTAARON891-71-5745FLUSHINGM2128679131
101,AARON,BRENT,M,891-71-5745,FLUSHING,6094834545,BRENTAARON891-71-5745FLUSHINGM6094834545


In [92]:
matching_data.iloc[[106,103],:]

,LAST,FIRST,GENDER,SSN,CITY,PHONE,BLOCK_SN
106,AARON,REECE,M,879-27-4618,NEW YORK,9592644070,REECEAARON879-27-4618NEW YORKM9592644070
103,AARON,REECE,M,879-27-4618,NaN,2672569637,REECEAARON879-27-4618nanM2672569637


In [93]:
matching_data.iloc[[2494,2493],:]

,LAST,FIRST,GENDER,SSN,CITY,PHONE,BLOCK_SN
2494,ABUSUFAIT,BREANA,U,807-66-1099,BROOKLYN,8484033472,BREANAABUSUFAIT807-66-1099BROOKLYNU8484033472
2493,ABUSUFAIT,BREANA,F,807-66-1099,BROOKLYN,5707997574,BREANAABUSUFAIT807-66-1099BROOKLYNF5707997574


In [112]:
matching_data.iloc[[2331,2321],:]

,LAST,FIRST,GENDER,SSN,CITY,PHONE,BLOCK_SN
2331,ABRPLE,CATHERINE,F,896-30-8100,BROOKLYN,NaN,CATHERINEABRPLE896-30-8100BROOKLYNFnan
2321,ABROLE,CATHERINE,F,896-30-8100,BROOKLYN,7328624547,CATHERINEABROLE896-30-8100BROOKLYNF7328624547
